# download dataset

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Sebercheres/Machine_Learning_Final_Project/main/Dataset/water-potability.csv', index_col=0)

# define scoring dataset

In [2]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import precision_score

def score_dataset(X_train, X_valid, y_train, y_valid):
  dum = DummyClassifier(random_state=7)
  dum.fit(X_train, y_train)
  predict = dum.predict(X_valid)
  prec = precision_score(y_valid, predict,average='macro')
  return prec

# define splitting data

In [3]:
from sklearn.model_selection import train_test_split

def split_data(data):
  X = data.drop('Potability', axis=1)
  y = data['Potability']
  X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, stratify=y, random_state=7)
  return X_train, X_valid, y_train, y_valid

# check empty row


In [4]:
df.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

we will try two dataset which one is imputing the empty dataset and the other is to drop the missing row and we compare which is better

because of the sensitivity of water i like dropping the row method morethan imputing because it will result on bad predictions

In [5]:
df_drop = df.dropna()

In [6]:
df.shape

(3276, 10)

In [7]:
df_drop.shape

(2011, 10)

In [8]:
X_train, X_valid, y_train, y_valid = split_data(df_drop)
score_dataset(X_train, X_valid, y_train, y_valid)

print('dropped precision score: ', end="")
print(score_dataset(X_train, X_valid, y_train, y_valid))

dropped precision score: 0.4890406036651096


/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


we loss about 1000 data

the second one is we are going to try imputing the missing data with simple imputer.

In [9]:
df_impute = df.copy()

split the data

In [10]:
X_train, X_valid, y_train, y_valid = split_data(df_impute)

In [11]:
X_train.isnull().sum()

ph                 400
Hardness             0
Solids               0
Chloramines          0
Sulfate            622
Conductivity         0
Organic_carbon       0
Trihalomethanes    137
Turbidity            0
dtype: int64

we'll try to see what is the precsision if the data isn't imputed.

In [12]:
print('dont do anything precision score: ', end="")
print(score_dataset(X_train, X_valid, y_train, y_valid))

dont do anything precision score: 0.48973782106312225


/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


we'll try to impute the data.

In [13]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [14]:
print('imputed data precision score: ', end="")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

imputed data precision score: 0.48973782106312225


/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


now we'll try to scale the data

In [15]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
features= imputed_X_train.columns
imputed_X_train[features] = ss.fit_transform(imputed_X_train[features])
imputed_X_valid[features]=  ss.fit_transform(imputed_X_valid[features])

In [16]:
print('imputed data precision score with scaling: ', end="")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

imputed data precision score with scaling: 0.48973782106312225


/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


ok the conclusion is we are going to use imputed data with standard scaler

In [17]:
imputed_X_train.to_csv('X_train.csv')
imputed_X_valid.to_csv('X_valid.csv')
y_train.to_csv('y_train.csv')
y_valid.to_csv('y_valid.csv')